In [2]:
!pip install sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [1]:

import nest_asyncio
nest_asyncio.apply()

In [6]:
import sqlite3

Base working code that allows date filtering

In [14]:
from pyrogram import Client
import pandas as pd
from datetime import datetime

api_id = 26048155
api_hash = "907f85401aa5d3330edaba35908c817d"
app = Client("my_account", api_id, api_hash)

chat_id = "7117421425"

# Specify the start and end dates for filtering
start_date = datetime(2024, 5, 19)
end_date = datetime(2024, 5, 20)

async def main():
    messages = []

    async with app:
        async for message in app.get_chat_history(chat_id):
            message_date = message.date

            # Filter messages by date
            if start_date <= message_date <= end_date:
                messages.append({
                    "text": message.text,
                    "date": message_date,
                    "chat_id": message.chat.id,
                    "chat_name": message.chat.title or message.chat.first_name
                })

    # Create a DataFrame
    df = pd.DataFrame(messages)
    print(df)

app.run(main())


                        text                date     chat_id chat_name
0  Where the bitches at dawg 2024-05-19 16:14:01  7117421425    mkamil
1         lololololoolololol 2024-05-19 16:06:19  7117421425    mkamil
2                    testing 2024-05-19 16:06:16  7117421425    mkamil
3                    testing 2024-05-19 16:06:14  7117421425    mkamil


SQLite integration with above code

In [2]:
from pyrogram import Client
import pandas as pd
from datetime import datetime
import sqlite3

In [3]:
api_id = 26048155
api_hash = "907f85401aa5d3330edaba35908c817d"
app = Client("my_account", api_id, api_hash)

In [4]:
chat_id = "7117421425"

In [5]:
start_date = datetime(2024, 5, 18)
end_date = datetime(2024, 5, 20)


In [6]:
db_name = 'messages.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

In [7]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS messages (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        text TEXT,
        date TEXT,
        chat_id INTEGER,
        chat_name TEXT
    )
''')
conn.commit()

In [9]:

async def main():
    messages = []

    async with app:
        async for message in app.get_chat_history(chat_id):
            message_date = message.date

            # Filter messages by date
            if start_date <= message_date <= end_date:
                messages.append({
                    "text": message.text,
                    "date": message_date,
                    "chat_id": message.chat.id,
                    "chat_name": message.chat.title or message.chat.first_name
                })

    # Create a DataFrame
    df = pd.DataFrame(messages)

    # Insert data into SQLite
    for _, row in df.iterrows():
        cursor.execute('''
            INSERT INTO messages (text, date, chat_id, chat_name) VALUES (?, ?, ?, ?)
        ''', (row['text'], row['date'].strftime('%Y-%m-%d %H:%M:%S'), row['chat_id'], row['chat_name']))


    # Commit the transaction and close the connection
    conn.commit()
    conn.close()

    # Print DataFrame (optional)
    print(df)

app.run(main())

                        text                date     chat_id chat_name
0  Where the bitches at dawg 2024-05-19 16:14:01  7117421425    mkamil
1         lololololoolololol 2024-05-19 16:06:19  7117421425    mkamil
2                    testing 2024-05-19 16:06:16  7117421425    mkamil
3                    testing 2024-05-19 16:06:14  7117421425    mkamil
